In [60]:
import pandas as pd
import sys
sys.path.insert(0, '/users/swang/downloads/')
from config_g_key import gkey
import json
import requests
import time
import mpu
from math import sin, cos, sqrt, atan2, radians

# Step1: Calculate the Distance per trip

In [ ]:
fp1 = 'Bikeshare Ridership (2017 Q1).csv'
fp2 = 'Bikeshare Ridership (2017 Q2).csv'
fp3 = 'Bikeshare Ridership (2017 Q3).csv'
fp4 = 'Bikeshare Ridership (2017 Q4).csv'

df1 = pd.read_csv(fp1); df2 = pd.read_csv(fp2); df3 = pd.read_csv(fp3); df4 = pd.read_csv(fp4);
df1.info()

In [ ]:
df = df1.merge(df2, how='outer')
df = df.merge(df3, how='outer')
df = df.merge(df4, how='outer')
df.info()
df.head()

In [ ]:
# using google API to obtain the lat & lng for different stations

# combine the dataframe to get unique station name
station_1 = df['to_station_name'].rename(columns={"to_station_name":"Station_Name"})
station_2 = df['from_station_name'].rename(columns={"from_station_name":"Station_Name"})

station = pd.concat([station_1, station_1]).unique()

# Access Google API
lats = []
lngs = []
stations = []

#for x in station:
    try:
        url_geo = f"https://maps.googleapis.com/maps/api/geocode/json?address={x}&key={gkey}"
        response_geo = requests.get(url_geo)
        dict_geo = response_geo.json()

        lat = dict_geo['results'][0]['geometry']['location']['lat']
        lng = dict_geo['results'][0]['geometry']['location']['lng']
        
        stations.append(x)
        lats.append(lat)
        lngs.append(lng)

        print(f"Processing {x}, the lat is {lat} and lng is {lng}")
        time.sleep(1.05)
    
    except:
        print("Not Found.")
        
    continue
        
    

In [ ]:
# export API result to new Dataframe
stations_df = pd.DataFrame({"Stations": stations, "Lats" : lats, "Lngs": lngs})

In [61]:
# read the stations_accordinates dataframe
cd_df = pd.read_csv('stations_accordinates.csv')
cd_df.head()
df.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,from_station_id,from_station_name,to_station_id,to_station_name,user_type
0,712382,1/1/2017 0:00,1/1/2017 0:03,223,7051.0,Wellesley St E / Yonge St Green P,7089.0,Church St / Wood St,Member
1,712383,1/1/2017 0:00,1/1/2017 0:05,279,7143.0,Kendal Ave / Bernard Ave,7154.0,Bathurst Subway Station,Member
2,712384,1/1/2017 0:05,1/1/2017 0:29,1394,7113.0,Parliament St / Aberdeen Ave,7199.0,College St W / Markham St,Member
3,712385,1/1/2017 0:07,1/1/2017 0:21,826,7077.0,College Park South,7010.0,King St W / Spadina Ave,Member
4,712386,1/1/2017 0:08,1/1/2017 0:12,279,7079.0,McGill St / Church St,7047.0,University Ave / Gerrard St W,Member


In [62]:
# merge dataframe to get the coordinate for each station and form a new customized dataframe

new_df = pd.merge(df, cd_df, left_on="from_station_name", right_on="Stations", how='left')
merge1 = new_df.rename(columns={"Lats":"from_station_lats", "Lngs": "from_station_lngs"})
merge2 = pd.merge(merge1, cd_df, left_on="to_station_name", right_on="Stations", how='left')
new_df = merge2.rename(columns={"Lats":"to_station_lats", "Lngs": "to_station_lngs"})
new_df = new_df[["trip_id", "trip_start_time", "trip_stop_time", "trip_duration_seconds", "user_type",\
                "from_station_name", "from_station_lats", "from_station_lngs",\
                "to_station_name", "to_station_lats", "to_station_lngs" ]]
new_df.head()

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,from_station_name,from_station_lats,from_station_lngs,to_station_name,to_station_lats,to_station_lngs
0,712382,1/1/2017 0:00,1/1/2017 0:03,223,Member,Wellesley St E / Yonge St Green P,43.667577,-79.384727,Church St / Wood St,34.159063,-99.291238
1,712383,1/1/2017 0:00,1/1/2017 0:05,279,Member,Kendal Ave / Bernard Ave,43.671929,-79.408432,Bathurst Subway Station,43.666001,-79.411215
2,712384,1/1/2017 0:05,1/1/2017 0:29,1394,Member,Parliament St / Aberdeen Ave,43.665045,-79.368281,College St W / Markham St,34.750263,-92.322501
3,712385,1/1/2017 0:07,1/1/2017 0:21,826,Member,College Park South,34.752938,-79.489493,King St W / Spadina Ave,43.645488,-79.394895
4,712386,1/1/2017 0:08,1/1/2017 0:12,279,Member,McGill St / Church St,35.412289,-80.668094,University Ave / Gerrard St W,43.657530,-79.389626


In [71]:
# Clean the Data
new_df_2 = new_df.dropna(subset=['from_station_lats', 'from_station_lngs', 'to_station_lats', 'to_station_lngs'], how='any')
new_df_2.reset_index(drop=True, inplace=True)
new_df_2.head(5)

,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,from_station_name,from_station_lats,from_station_lngs,to_station_name,to_station_lats,to_station_lngs
0,712382,1/1/2017 0:00,1/1/2017 0:03,223,Member,Wellesley St E / Yonge St Green P,43.667577,-79.384727,Church St / Wood St,34.159063,-99.291238
1,712383,1/1/2017 0:00,1/1/2017 0:05,279,Member,Kendal Ave / Bernard Ave,43.671929,-79.408432,Bathurst Subway Station,43.666001,-79.411215
2,712384,1/1/2017 0:05,1/1/2017 0:29,1394,Member,Parliament St / Aberdeen Ave,43.665045,-79.368281,College St W / Markham St,34.750263,-92.322501
3,712385,1/1/2017 0:07,1/1/2017 0:21,826,Member,College Park South,34.752938,-79.489493,King St W / Spadina Ave,43.645488,-79.394895
4,712386,1/1/2017 0:08,1/1/2017 0:12,279,Member,McGill St / Church St,35.412289,-80.668094,University Ave / Gerrard St W,43.657530,-79.389626


In [91]:
distances = []
R = 6373.0

lat1 = new_df_2['from_station_lats']
lon1 = new_df_2['from_station_lngs']

lat2 = new_df_2['to_station_lats']
lon2 = new_df_2['to_station_lngs']


for i in range(len(lat1)):
    lat_1 = radians(lat1[i])
    lon_1 = radians(lon1[i])
    lat_2 = radians(lat2[i])
    lon_2 = radians(lon2[i])

    dlon = lon_2 - lon_1
    dlat = lat_2 - lat_1
    
    a = sin(dlat / 2)**2 + cos(lat_1) * cos(lat_2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    dist = R * c
    distances.append(dist)

new_df_2["Distances"] = distances
new_df_2.to_csv('distances.csv')

/Users/swang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Step2: Merging the data to conduct study of,
1.	How do temperatures and precipitation levels impact bike share trip distance for non-members
2.	How do temperatures and precipitation levels impact bike share trip distance for non-members

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib notebook

In [132]:
f1 = 'Station_Distance.csv'
f2 = 'Toronto_Daily_Weather_2017.csv'

df1 = pd.read_csv(f1); df2 = pd.read_csv(f2)

In [133]:
df1.head()

,Unnamed: 0,trip_id,trip_start_time,trip_stop_time,trip_duration_seconds,user_type,from_station_name,from_station_lats,from_station_lngs,to_station_name,to_station_lats,to_station_lngs,Distances
0,0,712382,1/1/2017 0:00,1/1/2017 0:03,223,Member,Wellesley St E / Yonge St Green P,43.667577,-79.384727,Church St / Wood St,34.159063,-99.291238,2013.655908
1,1,712383,1/1/2017 0:00,1/1/2017 0:05,279,Member,Kendal Ave / Bernard Ave,43.671929,-79.408432,Bathurst Subway Station,43.666001,-79.411215,0.696384
2,2,712384,1/1/2017 0:05,1/1/2017 0:29,1394,Member,Parliament St / Aberdeen Ave,43.665045,-79.368281,College St W / Markham St,34.750263,-92.322501,1490.010191
3,3,712385,1/1/2017 0:07,1/1/2017 0:21,826,Member,College Park South,34.752938,-79.489493,King St W / Spadina Ave,43.645488,-79.394895,989.150183
4,4,712386,1/1/2017 0:08,1/1/2017 0:12,279,Member,McGill St / Church St,35.412289,-80.668094,University Ave / Gerrard St W,43.657530,-79.389626,923.616703


In [134]:
# remove the time, keep only date
df_date = df1['trip_start_time'].str.split(expand=True)
df_date = df_date.rename(columns={0:"Date", 1:"Time"})
df_date.head()

# form a new data
df_new = df_date.join(df1)
df_new = df_new[["Date", "user_type", "Distances", "trip_duration_seconds"]]
df_new.head()

,Date,user_type,Distances,trip_duration_seconds
0,1/1/2017,Member,2013.655908,223
1,1/1/2017,Member,0.696384,279
2,1/1/2017,Member,1490.010191,1394
3,1/1/2017,Member,989.150183,826
4,1/1/2017,Member,923.616703,279


In [135]:
# change "Date" format
df_new['Date_Dist'] = pd.to_datetime(df_new.Date, errors="coerce")

# Reorganize the dateframe
df_date = df_new[["Date_Dist", "user_type", "Distances", "trip_duration_seconds"]]
df_date.head()

In [137]:
#dropping abnormal distance
condition_dis = (df_date['Distances']>0) & (df_date['Distances']<=10)
df_new_dis = df_date.loc[condition_dis].rename(columns={"Date_Dist":"Date", "Distances": "ave_distance"})
df_new_dis.head()

,Date,user_type,ave_distance,trip_duration_seconds
1,2017-01-01,Member,0.696384,279
6,2017-01-01,Member,0.871079,307
7,2017-01-01,Member,1.963143,647
8,2017-01-01,Member,0.544524,217
9,2017-01-01,Member,1.158759,467


In [138]:
# Group dateframe by 'Date' and 'user_type' and cal the mean
user_avg_distance = df_new_dis.groupby(['Date', 'user_type']).mean()
user_avg_distance.to_csv("user_distance_avg_daily.csv")
user_avg_distance.head()

ave_distance  trip_duration_seconds
Date       user_type                                     
2017-01-01 Casual         1.646176            1914.307692
           Member         1.527531             646.953020
2017-01-02 Casual         1.898854            2559.750000
           Member         1.674482             652.883895
2017-01-03 Casual         1.954056            1433.423077

In [139]:
# change the 'Toronto_Daily_Weather_2017' Date format 
df2['Date'] = pd.to_datetime(df2.Date)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 5 columns):
Unnamed: 0      365 non-null int64
Date            365 non-null datetime64[ns]
avg_temp        365 non-null float64
avg_rain        365 non-null float64
weather_main    365 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 14.3+ KB


In [150]:
# merge the "user_avg_distance" and the "Toronto_Daily_Weather_2017"

df_all = pd.merge(df2, user_avg_distance, left_on="Date", right_on="Date", how='left', left_index=True).reset_index("user_type")
df_all.head()

,user_type,Unnamed: 0,Date,avg_temp,avg_rain,weather_main,ave_distance,trip_duration_seconds
0,Casual,0,2017-01-01,274.440741,0.088889,Clear,1.646176,1914.307692
1,Member,0,2017-01-01,274.440741,0.088889,Clear,1.527531,646.953020
2,Casual,1,2017-01-02,273.157083,0.000000,Clouds,1.898854,2559.750000
3,Member,1,2017-01-02,273.157083,0.000000,Clouds,1.674482,652.883895
4,Casual,2,2017-01-03,276.621731,0.305769,Mist,1.954056,1433.423077


In [151]:
# for any null value in "Distances" and "trip_duration_seconds", fill in a mean value
dist_mean = df_all['ave_distance'].mean()
trip_dura_mean = df_all['trip_duration_seconds'].mean()

df_all['ave_distance'] = df_all['ave_distance'].fillna(value=dist_mean)
df_all['trip_duration_seconds'] = df_all['trip_duration_seconds'].fillna(value=dist_mean)
df_all.columns

# re-order the dataframe
df_all = df_all[["Date", "user_type", "avg_temp", "avg_rain", "ave_distance", "weather_main", "trip_duration_seconds"]]
df_all.to_csv('weather_distance_avg_daily_2017.csv')
df_all.head()

,Date,user_type,avg_temp,avg_rain,ave_distance,weather_main,trip_duration_seconds
0,2017-01-01,Casual,274.440741,0.088889,1.646176,Clear,1914.307692
1,2017-01-01,Member,274.440741,0.088889,1.527531,Clear,646.953020
2,2017-01-02,Casual,273.157083,0.000000,1.898854,Clouds,2559.750000
3,2017-01-02,Member,273.157083,0.000000,1.674482,Clouds,652.883895
4,2017-01-03,Casual,276.621731,0.305769,1.954056,Mist,1433.423077
